In [1]:
#Import the dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy import stats
import os

In [2]:
# Load dataset 
#df = pd.read_csv('data/benin_raw.csv')
df=pd.read_csv('F:/solar-challenge-week1/data/benin_raw.csv')

In [ ]:
# for the visualization purpose 
plt.style.use('seaborn')
sns.set_context("notebook", font_scale=1.2)


In [ ]:
#df.head(5)

In [3]:
#Summary Statistics & Missing-Value Report
#Overview of numeric columns
print("The description of the numeric columns:")
print(df.describe())
#Overview of categorical columns
print("***************************")
print("The description of the categorical columns: ")
print(df.describe(include=['object']))
#Overview of all columns
print("****************************")
print("The description of all columns:")
print (df.describe(include='all'))
#Overview of the dataset
print("****************************")
print("An overview of the dataset: the first 5 rows") 
print(df.head())
#Overview of the dataset
print("****************************")
print("An overview of the dataset: the last 5 rows")

# Overview of numeric columns
print("The description of the numeric columns:") 
print (df.describe())
#Overview of categorical columns
print("****************************")
print("The description of the categorical columns:") 
print(df.describe(include=['object']))
#Overview of all columns
print("****************************")
print("The description of all columns:")
print (df.describe(include='all'))
#Overview of the dataset
print("****************************")
print("An overview of the dataset: the first 5 rows") 
print(df.head())
#Overview of the dataset
print("****************************")
print("An overview of the dataset: the last 5 rows")
print(df.tail())
#Overview of the dataset
print("****************************")
print("An overview of the dataset: a random sample of 5 rows")
print(df.sample())
#Overview of the dataset
print("****************************")
print("An overview of the dataset: the shape of the dataset")
print(df.info())
#Check for missing values
print("****************************")
print("Check for missing values: ")
print(df.isna().sum())


The description of the numeric columns:
                 GHI            DNI            DHI           ModA  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      240.559452     167.187516     115.358961     236.589496   
std       331.131327     261.710501     158.691074     326.894859   
min       -12.900000      -7.800000     -12.600000       0.000000   
25%        -2.000000      -0.500000      -2.100000       0.000000   
50%         1.800000      -0.100000       1.600000       4.500000   
75%       483.400000     314.200000     216.300000     463.700000   
max      1413.000000     952.300000     759.200000    1342.300000   

                ModB           Tamb             RH             WS  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      228.883576      28.179683      54.487969       2.121113   
std       316.536515       5.924297      28.073069       1.603466   
min         0.000000      11.000000       2.100000       0.000

In [4]:
# 2. Outlier Detection & Basic Cleaning
key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
print("\n=== Outlier Detection (Z-scores > 3) ===")
for col in key_columns:
    df[f'{col}_zscore'] = np.abs(stats.zscore(df[col], nan_policy='omit'))
outliers = df[df[[f'{col}_zscore' for col in key_columns]].gt(3).any(axis=1)]
print(f"Number of rows with outliers (|Z|>3): {len(outliers)}")


=== Outlier Detection (Z-scores > 3) ===
Number of rows with outliers (|Z|>3): 7740


In [ ]:
#mpute missing values in key columns with median
for col in key_columns:
    df[col].fillna(df[col].median(), inplace=True)


In [ ]:
# Export cleaned DataFrame
if not os.path.exists('output'):
    os.makedirs('output')
df.drop(columns=[f'{col}_zscore' for col in key_columns]).to_csv('output/benin_clean.csv', index=False)
print("\nCleaned data exported to data/benin_clean.csv")